### Install depends:

In [ ]:
# Install TensorFlow (Kaggle default images usually have it pre-installed, but just in case)
!pip install -q tensorflow tqdm opencv-python-headless matplotlib tensorflow-addons


### Train the TF model to predict mtg corner locations using the synthetic dataset:

In [ ]:
import os
import json
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers
from tensorflow.keras.applications import MobileNetV2

# === Config ===
INPUT_DIR        = "/kaggle/input/mtg-keypoint-regression-dataset-5k"
ANNOTATION_FILE  = os.path.join(INPUT_DIR, "dataset", "dataset", "annotations.json")
IMAGE_DIR        = os.path.join(INPUT_DIR, "dataset", "dataset")
OUTPUT_DIR       = "/kaggle/working"
VISUALIZE_DIR    = os.path.join(OUTPUT_DIR, "training_visualizations")
MODEL_PATH       = os.path.join(OUTPUT_DIR, "mtg_keypoint_tfjs_safe_model")

IMAGE_SIZE       = 224
HEATMAP_SIZE     = 56
SIGMA_PIXELS     = 1.5
BATCH_SIZE       = 16
EPOCHS           = 35
ORIGINAL_DIM     = 1024

os.makedirs(VISUALIZE_DIR, exist_ok=True)

# === Load and validate annotations ===
with open(ANNOTATION_FILE, "r") as f:
    raw = json.load(f)

def valid(a):
    pts = a.get("corners", [])
    return (isinstance(pts, list) and len(pts) == 4 and
            all(isinstance(x, (int,float)) and isinstance(y, (int,float)) and
                0 <= x <= ORIGINAL_DIM and 0 <= y <= ORIGINAL_DIM
                for x,y in pts))

annotations = [a for a in raw if valid(a)]
split = int(0.9 * len(annotations))
train_anns, val_anns = annotations[:split], annotations[split:]

# === Precompute grid for heatmaps ===
xs = np.arange(HEATMAP_SIZE)
ys = np.arange(HEATMAP_SIZE)
grid_x, grid_y = np.meshgrid(xs, ys)

def make_heatmaps(corners):
    hm = np.zeros((HEATMAP_SIZE, HEATMAP_SIZE, 4), dtype=np.float32)
    for i, (x_o, y_o) in enumerate(corners):
        x = (x_o / ORIGINAL_DIM) * (HEATMAP_SIZE - 1)
        y = (y_o / ORIGINAL_DIM) * (HEATMAP_SIZE - 1)
        d2 = (grid_x - x)**2 + (grid_y - y)**2
        hm[..., i] = np.exp(-d2 / (2 * SIGMA_PIXELS**2))
    return hm

def data_gen(anns):
    while True:
        imgs = np.zeros((BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        hmts = np.zeros((BATCH_SIZE, HEATMAP_SIZE, HEATMAP_SIZE, 4), dtype=np.float32)
        coords = np.zeros((BATCH_SIZE, 4, 2), dtype=np.float32)
        for i in range(BATCH_SIZE):
            a = np.random.choice(anns)
            img = cv2.imread(os.path.join(IMAGE_DIR, a["filename"]))
            img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE)).astype(np.float32) / 255.0
            hm = make_heatmaps(a["corners"])
            imgs[i]   = img
            hmts[i]   = hm
            coords[i] = np.array(a["corners"], dtype=np.float32) / ORIGINAL_DIM
        yield imgs, (hmts, coords)

train_ds = tf.data.Dataset.from_generator(
    lambda: data_gen(train_anns),
    output_signature=(
        tf.TensorSpec((BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), tf.float32),
        (tf.TensorSpec((BATCH_SIZE, HEATMAP_SIZE, HEATMAP_SIZE, 4), tf.float32),
         tf.TensorSpec((BATCH_SIZE, 4, 2), tf.float32))
    )
).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_generator(
    lambda: data_gen(val_anns),
    output_signature=(
        tf.TensorSpec((BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), tf.float32),
        (tf.TensorSpec((BATCH_SIZE, HEATMAP_SIZE, HEATMAP_SIZE, 4), tf.float32),
         tf.TensorSpec((BATCH_SIZE, 4, 2), tf.float32))
    )
).prefetch(tf.data.AUTOTUNE)

@tf.keras.utils.register_keras_serializable()
class SoftArgmax(layers.Layer):
    def __init__(self, H, W, **kw):
        super().__init__(**kw)
        xs = tf.linspace(0.0, 1.0, W)
        ys = tf.linspace(0.0, 1.0, H)
        gx, gy = tf.meshgrid(xs, ys)
        self.grid_x = tf.reshape(gx, [-1])
        self.grid_y = tf.reshape(gy, [-1])
        self.H, self.W = H, W

    def call(self, heatmaps):
        B = tf.shape(heatmaps)[0]
        flat = tf.reshape(heatmaps, [B, self.H*self.W, 4])
        probs = tf.nn.softmax(flat, axis=1)
        exp_x = tf.reduce_sum(probs * self.grid_x[None,:,None], axis=1)
        exp_y = tf.reduce_sum(probs * self.grid_y[None,:,None], axis=1)
        return tf.stack([exp_x, exp_y], axis=-1)

def build_model():
    inp = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")
    base = MobileNetV2(include_top=False, input_tensor=inp, weights='imagenet')
    x = base.output
    x = layers.Conv2DTranspose(128, 3, strides=2, padding='same',
                               activation=layers.ReLU(max_value=6.0),
                               use_bias=False,
                               kernel_regularizer=regularizers.l2(1e-5))(x)
    x = layers.SeparableConv2D(96, 3, padding='same',
                               activation=layers.ReLU(max_value=6.0),
                               use_bias=False,
                               depthwise_regularizer=regularizers.l2(1e-5),
                               pointwise_regularizer=regularizers.l2(1e-5))(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.SeparableConv2D(64, 3, padding='same',
                               activation=layers.ReLU(max_value=6.0),
                               use_bias=False,
                               depthwise_regularizer=regularizers.l2(1e-5),
                               pointwise_regularizer=regularizers.l2(1e-5))(x)
    x = layers.UpSampling2D(2)(x)
    feat = layers.SeparableConv2D(64, 3, padding='same',
                                  activation=layers.ReLU(max_value=6.0),
                                  use_bias=False,
                                  depthwise_regularizer=regularizers.l2(1e-5),
                                  pointwise_regularizer=regularizers.l2(1e-5))(x)
    heatmaps = layers.Conv2D(4, 1, padding='same', activation=None, use_bias=False, name='heatmaps')(feat)
    coords = SoftArgmax(HEATMAP_SIZE, HEATMAP_SIZE, name='coords')(heatmaps)
    return models.Model(inputs=inp, outputs=[heatmaps, coords])

model = build_model()

def corner_error(y_true, y_pred):
    err = tf.norm((y_true - y_pred) * ORIGINAL_DIM, axis=-1)
    return tf.reduce_mean(err)

model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss={'heatmaps': 'mse', 'coords': 'mse'},
    loss_weights={'heatmaps': 1.0, 'coords': 0.0},
    metrics={'coords': corner_error}
)

model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=len(train_anns) // BATCH_SIZE,
    validation_steps=len(val_anns) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[
        callbacks.EarlyStopping(
            monitor='val_coords_corner_error',
            mode='min',
            patience=3,
            restore_best_weights=True
        )
    ]
)

# Export only the coordinate output model for TF.js deployment
model.export(MODEL_PATH)
print("✅ WASM-compatible SavedModel exported to:", MODEL_PATH)

In [ ]:
model.export(MODEL_PATH)

## Convert to tensorflowjs for use on the web:

In [ ]:
!pip install tensorflowjs

### Convert the Keras model to tfjs graph model for use on the web:

In [ ]:
#!/usr/bin/env python3

import os
import shutil

# ─── CONFIG ──────────────────────────────────────────────────────────────────────
SAVEDMODEL_DIR = "/kaggle/working/mtg_keypoint_tfjs_safe_model"  # No .h5 here!
TFJS_OUT_DIR   = "/kaggle/working/web_model"

# ─── CLEAN ───────────────────────────────────────────────────────────────────────
if not os.path.isdir(SAVEDMODEL_DIR):
    raise FileNotFoundError(f"❌ SavedModel not found at: {SAVEDMODEL_DIR}")

shutil.rmtree(TFJS_OUT_DIR, ignore_errors=True)
os.makedirs(TFJS_OUT_DIR, exist_ok=True)

# ─── CONVERT TO TFJS (with Float16 Quantization) ─────────────────────────────────
print("🚀 Converting SavedModel to TensorFlow.js format with Float16 quantization...")
exit_code = os.system(
    f"tensorflowjs_converter "
    f"--input_format=tf_saved_model "
    f"--output_format=tfjs_graph_model "
    f"--quantize_float16 "
    f"--strip_debug_ops=true "
    f"{SAVEDMODEL_DIR} "
    f"{TFJS_OUT_DIR}"
)

if exit_code != 0:
    raise RuntimeError(f"❌ TFJS conversion failed with exit code {exit_code}")

print(f"✅ Quantized TFJS GraphModel exported to: {TFJS_OUT_DIR}")

In [ ]:
import shutil
import os

# Paths
web_model_dir = os.path.join('/kaggle/working', 'web_model')
zip_path = os.path.join('/kaggle/working', 'web_model')

# Remove existing archive
if os.path.exists(f"{zip_path}.zip"):
    os.remove(f"{zip_path}.zip")

# Create ZIP archive
shutil.make_archive(zip_path, 'zip', web_model_dir)

print(f"✅ Created archive: {zip_path}.zip")
